# Obiettivo del Progetto

L’obiettivo di questo progetto è valutare le prestazioni di PostgreSQL eseguendo alcune query di selezione, proiezione e join in due configurazioni differenti:

1. **Senza indici**: oltre alle chiavi primarie di base.
2. **Con indici**: creati ad hoc su determinati campi.

Si confronteranno i piani di esecuzione (mediante `EXPLAIN ANALYZE`) e si misureranno, opzionalmente, i blocchi letti/hittati (pin e read) tramite la vista di sistema `pg_stat_database`.

---
### Generazione di Dati Fittizi con Faker

In questa sezione, utilizzeremo la libreria Python `faker` per generare dati fittizi. Questo può essere utile per simulare un database o per testare query SQL. Lo script seguente crea un file CSV con dati separati da un delimitatore personalizzato (ad esempio, `|`).


In [1]:
import psycopg2
from faker import Faker
import random
import subprocess
import os

In [2]:
import os
import subprocess

print("Current working directory:", os.getcwd())

def run_sql_script(script_path, db_name, db_user, db_password, db_host="localhost"):
    env = os.environ.copy()
    env["PGPASSWORD"] = db_password

    cmd = [
        "psql",
        "-U", db_user,
        "-d", db_name,
        "-h", db_host,
        "-f", script_path
    ]
    
    print(f"Esecuzione di {script_path}...")
    subprocess.run(cmd, check=True, env=env)
    print(f"{script_path} completato.\n")

if __name__ == "__main__":
    # Parametri di connessione
    DB_NAME = "universita"      # Sostituisci con il nome del tuo database
    DB_USER = "postgres"      # Sostituisci se necessario
    DB_PASSWORD = "postgres"  # Sostituisci con la tua password
    DB_HOST = "localhost"
    
    # Percorsi dei file SQL (in questo esempio, nella directory corrente)
    create_tables_path = os.path.join(os.getcwd(), "create_tables.sql")
    create_indexes_btree_path = os.path.join(os.getcwd(), "create_indexes_btree.sql")
    create_indexes_hash_path = os.path.join(os.getcwd(), "create_indexes_hash.sql")

Current working directory: c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2


### Configurazione A senza indici

In [3]:
# Per configurazione A (senza indice)
print("Configurazione A: Senza indice")
run_sql_script(create_tables_path, DB_NAME, DB_USER, DB_PASSWORD, DB_HOST)

Configurazione A: Senza indice
Esecuzione di c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2\create_tables.sql...
c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2\create_tables.sql completato.



### Configurazione B (Indice B-Tree su id_corso)

In [20]:
# Per configurazione B (con indice B-Tree)
print("Configurazione B: Con indice B-Tree")
run_sql_script(create_indexes_btree_path, DB_NAME, DB_USER, DB_PASSWORD, DB_HOST)

Configurazione B: Con indice B-Tree
Esecuzione di c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2\create_indexes_btree.sql...
c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2\create_indexes_btree.sql completato.



### Configurazione C (Indice Hash su corso)

In [21]:
# poi esegui l'indice Hash:
print("Configurazione C: Con indice Hash")
run_sql_script(create_indexes_hash_path, DB_NAME, DB_USER, DB_PASSWORD, DB_HOST)

Configurazione C: Con indice Hash
Esecuzione di c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2\create_indexes_hash.sql...
c:\Users\luca-\OneDrive\Desktop\University\MAGISTRALE\2_ANNO\2_SEMESTRE\Basi_di_Dati_2\Homeworks\Progetto1_bd2\create_indexes_hash.sql completato.



In [4]:

# Parametri di connessione al database
DB_NAME = "universita"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"  # o l'host appropriato

fake = Faker()

# Dimensioni dei dati
N_STUDENTI = 100000
N_CORSI = 50000
N_ISCRIZIONI = 10000

# Connessione al database
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST)
cur = conn.cursor()

# Svuota le tabelle (se esistono dati)
cur.execute("TRUNCATE TABLE Iscrizione CASCADE;")
cur.execute("TRUNCATE TABLE Corso CASCADE;")
cur.execute("TRUNCATE TABLE Studente CASCADE;")

# Popola la tabella Studente
for i in range(1, N_STUDENTI + 1):
    nome = fake.first_name()
    email = fake.email()
    indirizzo = fake.address().replace("\n", ", ")
    cur.execute(
        "INSERT INTO Studente (id, nome, email, indirizzo) VALUES (%s, %s, %s, %s);",
        (i, nome, email, indirizzo)
    )

# Popola la tabella Corso
for i in range(1, N_CORSI + 1):
    nome_corso = "Corso_" + fake.word()
    cfu = random.randint(3, 12)
    cur.execute(
        "INSERT INTO Corso (id, nome, cfu) VALUES (%s, %s, %s);",
        (i, nome_corso, cfu)
    )

# Popola la tabella Iscrizione
for _ in range(N_ISCRIZIONI):
    stud_id = random.randint(1, N_STUDENTI)
    corso_id = random.randint(1, N_CORSI)
    voto = random.randint(18, 30)
    cur.execute(
        "INSERT INTO Iscrizione (studente_id, corso_id, voto) VALUES (%s, %s, %s);",
        (stud_id, corso_id, voto)
    )

conn.commit()
cur.close()
conn.close()

print("Popolamento completato!")


Popolamento completato!


## Visualizzazione dati nelle relazioni

In [5]:
import psycopg2

# Connessione al database
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST)
cur = conn.cursor()

# Relazioni da visualizzare
relazioni = ["Studente", "Corso", "Iscrizione"]

for relazione in relazioni:
    print(f"Primi 10 record della tabella {relazione}:")
    cur.execute(f"SELECT * FROM {relazione} LIMIT 10;")
    records = cur.fetchall()
    for record in records:
        print(record)
    print("\n")

cur.close()
conn.close()


Primi 10 record della tabella Studente:
(1, 'Teresa', 'wsmith@example.com', '2475 Thomas Lock, New Kevintown, AR 37617')
(2, 'Tiffany', 'cindyrhodes@example.com', '7427 Ramirez Villages, Gordonside, AR 53341')
(3, 'Stanley', 'ericwong@example.net', 'PSC 6477, Box 8085, APO AA 88923')
(4, 'Kelly', 'gloriapratt@example.com', '52822 Gomez Haven, West Desiree, LA 15016')
(5, 'Mike', 'smithmatthew@example.net', '78411 Carlson Creek, Lake Jessica, MS 15786')
(6, 'Adam', 'jennifer25@example.net', '884 Carrie Plaza, Maynardport, NH 02765')
(7, 'Michael', 'taylormark@example.org', '226 Merritt Plains Apt. 121, South Andrew, ND 24516')
(8, 'Jeffrey', 'sharmon@example.net', '8965 Brenda Parkway Apt. 272, South Victorchester, PA 14879')
(9, 'Jacqueline', 'seannash@example.net', '07931 Martin Cliff Apt. 962, Cunninghamshire, ND 18234')
(10, 'Ashley', 'blevinsjohn@example.org', '19442 Dawn Plaza, Williamborough, HI 27210')


Primi 10 record della tabella Corso:
(1, 'Corso_every', 11)
(2, 'Corso_stan

# Parte 2 del Progetto

La seconda parte del progetto consiste nel definire ed eseguire le query di test per confrontare i piani di esecuzione e le prestazioni nelle diverse configurazioni (senza indice, con indice B-Tree e con indice Hash).

Questa funzione esegue i seguenti passaggi:

1. **Drop della tabella esistente**: Se esiste una tabella con lo stesso nome, viene eliminata.
2. **Creazione di una nuova tabella**: La tabella viene creata con i valori iniziali di `blks_hit + blks_read` e `blks_read`, letti dalla vista `pg_stat_database`.
3. **Esecuzione della query da testare**: La query viene eseguita per misurare le prestazioni.
4. **Attesa opzionale**: È possibile inserire un'attesa per stabilizzare le statistiche.
5. **Lettura dei valori finali**: I valori finali di `blks_hit` e `blks_read` vengono letti nuovamente da `pg_stat_database`.
6. **Calcolo delle differenze**: Viene calcolata la differenza tra i valori iniziali e finali per ottenere i risultati di `pin` (numPin) e `read` (numRead).
7. **Stampa dei risultati**: I valori risultanti vengono stampati a schermo.

**Nota importante**: Nel comando `WHERE datname LIKE 'postgres'`, sostituisci `'postgres'` con il nome del database che stai utilizzando (ad esempio, `'universita'` se il tuo database si chiama “universita”).


In [6]:
import psycopg2
import time

def test_query(query, db_name="universita", user="postgres", password="postgres", host="localhost"):
    """
    Esegue la query 'query' e mostra il numero di pin e letture fisiche 
    dovute a quella query, calcolate dalla differenza in pg_stat_database.
    """
    conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
    cur = conn.cursor()
    
    # 1) Rimuove la tabella temporanea 'prima' se già esiste
    # 2) Seleziona i valori attuali di blks_hit e blks_read e li inserisce in 'prima'
    cur.execute(f"""
        DROP TABLE IF EXISTS prima CASCADE;
        SELECT blks_hit + blks_read AS numPin, 
               blks_read AS numRead
        INTO prima
        FROM pg_stat_database
        WHERE datname LIKE '{db_name}';
    """)
    conn.commit()
    
    # 3) Esegue la query di test
    cur.execute(query)
    # Se la query restituisce dati (es: SELECT), li leggiamo per completare
    try:
        _ = cur.fetchall()
    except:
        pass
    
    conn.commit()
    
    # 4) (Opzionale) attende qualche secondo
    time.sleep(2)
    
    # 5) Calcola la differenza con i nuovi valori di pg_stat_database
    #    (dopo la query)
    cur.execute(f"""
        SELECT 
          ((dopo.blks_hit + dopo.blks_read) - prima.numPin) AS numPin,
          (dopo.blks_read - prima.numRead) AS numRead
        FROM pg_stat_database AS dopo, prima
        WHERE dopo.datname LIKE '{db_name}';
    """)
    conn.commit()
    
    results = cur.fetchall()  # Esempio: [(150, 12)]
    cur.close()
    conn.close()
    
    # 6) Stampa i risultati
    #    results è una lista di tuple, es. [(numPin, numRead)]
    if results:
        num_pin, num_read = results[0]
        print(f"Query eseguita: {query.strip()}")
        print(f"  --> Num Pin: {num_pin}, Num Read: {num_read}\n")
    else:
        print("Nessun risultato calcolato.")



In [33]:
# Parametri di connessione
DB_NAME = "universita"   # o "postgres", se il DB si chiama così
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"

# Definizione delle query
q1 = """
EXPLAIN ANALYZE
SELECT nome, email
FROM Studente
WHERE id = 500;
"""

q2 = """
EXPLAIN ANALYZE
SELECT SUM(C.cfu) AS somma_cfu
FROM Iscrizione I
JOIN Corso C ON I.corso_id = C.id
WHERE I.voto > 25;
"""

q3 = """
EXPLAIN ANALYZE
SELECT *
FROM Iscrizione
WHERE corso_id = 1234;
"""

# Esegui i test
test_query(q1, db_name=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST)
test_query(q2, db_name=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST)
test_query(q3, db_name=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST)

Query eseguita: EXPLAIN ANALYZE
SELECT nome, email
FROM Studente
WHERE id = 500;
  --> Num Pin: 0, Num Read: 0

Query eseguita: EXPLAIN ANALYZE
SELECT SUM(C.cfu) AS somma_cfu
FROM Iscrizione I
JOIN Corso C ON I.corso_id = C.id
WHERE I.voto > 25;
  --> Num Pin: 0, Num Read: 0

Query eseguita: EXPLAIN ANALYZE
SELECT *
FROM Iscrizione
WHERE corso_id = 1234;
  --> Num Pin: 0, Num Read: 0

